## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tasiilaq,GL,65.6145,-37.6368,33.96,71,100,14.03,overcast clouds
1,1,Sorland,NO,67.6670,12.6934,41.83,84,97,23.04,overcast clouds
2,2,Bredasdorp,ZA,-34.5322,20.0403,47.52,74,14,4.92,few clouds
3,3,Avarua,CK,-21.2078,-159.7750,78.85,73,90,11.50,overcast clouds
4,4,Hermanus,ZA,-34.4187,19.2345,51.91,66,22,4.21,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Avarua,CK,-21.2078,-159.7750,78.85,73,90,11.50,overcast clouds
9,9,Airai,TL,-8.9266,125.4092,65.25,76,97,3.06,light rain
10,10,Mananara,MG,-16.1667,49.7667,69.33,93,36,6.91,light rain
12,12,Tam Ky,VN,15.5667,108.4833,80.58,65,56,1.68,broken clouds
13,13,Sao Filipe,CV,14.8961,-24.4956,72.61,77,8,3.80,clear sky
14,14,Isangel,VU,-19.5500,169.2667,76.77,63,61,11.05,broken clouds
15,15,Liuhe,CN,42.2401,125.7172,70.54,56,14,7.29,few clouds
19,19,Sumbe,AO,-11.2061,13.8437,76.62,80,46,3.31,scattered clouds
20,20,Rikitea,PF,-23.1203,-134.9692,76.32,59,25,9.22,scattered clouds
21,21,Saint Anthony,US,45.0205,-93.2180,81.30,70,20,3.00,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_cities_df = preferred_cities_df.dropna()
clean_cities_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Avarua,CK,78.85,overcast clouds,-21.2078,-159.7750,
9,Airai,TL,65.25,light rain,-8.9266,125.4092,
10,Mananara,MG,69.33,light rain,-16.1667,49.7667,
12,Tam Ky,VN,80.58,broken clouds,15.5667,108.4833,
13,Sao Filipe,CV,72.61,clear sky,14.8961,-24.4956,
14,Isangel,VU,76.77,broken clouds,-19.5500,169.2667,
15,Liuhe,CN,70.54,few clouds,42.2401,125.7172,
19,Sumbe,AO,76.62,scattered clouds,-11.2061,13.8437,
20,Rikitea,PF,76.32,scattered clouds,-23.1203,-134.9692,
21,Saint Anthony,US,81.30,few clouds,45.0205,-93.2180,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Not found... Skipping")
        

Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping


In [21]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   303
Country                303
Max Temp               303
Current Description    303
Lat                    303
Lng                    303
Hotel Name             303
dtype: int64

In [22]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [38]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
></dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [45]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))